# **Market Prediction Using Macro Economic Data**
using LSTM and TFT

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

---
---
---

# Importing and cleaning  data

## **"niftyvix"** 
is a completely preprocessed data and no changes are to be made here in terms of data processing

In [2]:
df0 = pd.read_csv("MacroData/finalniftyvix_data.csv")
niftyvix = df0.copy()
niftyvix.head()

,Date,Date.1,vixPrice,vixChange%,niftyPrice,niftyChange %,n5day,n10day,n20day,n1day,...,December,January,February,n1day%,n5day%,n10day%,n20day%,n60day%,cluster,Month
0,2008-06-05,2008-06-05,30.32,1.68,4676.95,1.99,4835.3,5025.45,5135.50,4647.00,...,0,0,0,-0.640375,3.385754,7.451437,9.804467,4.004747,2,6
1,2008-06-06,2008-06-06,30.32,0.00,4627.80,-1.05,4870.1,4946.55,5081.70,4761.20,...,0,0,0,2.882579,5.235749,6.887722,9.808116,6.344267,2,6
2,2008-06-09,2008-06-09,32.43,6.96,4500.95,-2.74,4739.6,4875.05,4982.60,4709.65,...,0,0,0,4.636799,5.302214,8.311579,10.701074,6.013175,2,6
3,2008-06-10,2008-06-10,30.23,-6.78,4449.80,-1.14,4715.9,4859.80,5012.65,4747.05,...,0,0,0,6.680076,5.980044,9.213897,12.648883,7.879006,2,6
4,2008-06-11,2008-06-11,29.64,-1.95,4523.60,1.66,4585.6,4918.35,4957.80,4733.00,...,0,0,0,4.629057,1.370590,8.726457,9.598550,7.566982,2,6


There is an extra date column so we will drop it

In [3]:
niftyvix = niftyvix.drop("Date.1", axis=1)
niftyvix.head(1)

,Date,vixPrice,vixChange%,niftyPrice,niftyChange %,n5day,n10day,n20day,n1day,n60day,...,December,January,February,n1day%,n5day%,n10day%,n20day%,n60day%,cluster,Month
0,2008-06-05,30.32,1.68,4676.95,1.99,4835.3,5025.45,5135.5,4647.0,4864.25,...,0,0,0,-0.640375,3.385754,7.451437,9.804467,4.004747,2,6


In [4]:
niftyvix[['Date']].head(1)

,Date
0,2008-06-05


## Above dataframe will be used as a template to clean and process data

> Dates must be in ascending order, hence DFs are being reciprocaled as necessary

> All date formats are being matched with "niftyvix" DF

> Copy of each orignal df is being made so that it stays safe

> All dates are changed from object to datetime format

---

## **"Gold"** 

In [5]:
df1 = pd.read_csv("MacroData/Gold.csv")
gold = df1.copy()
gold.head()

,Date,GoldPrice,GoldOpen,GoldHigh,GoldLow,Vol.,GoldChange %
0,27-08-2024,"2,529.30","2,530.50","2,537.70","2,515.40",7.27K,-0.08%
1,26-08-2024,"2,531.40","2,522.10","2,538.90","2,520.70",6.08K,0.35%
2,23-08-2024,"2,522.60","2,497.20","2,530.40","2,497.00",6.90K,1.17%
3,22-08-2024,"2,493.50","2,525.30","2,528.00","2,483.20",7.69K,-1.21%
4,21-08-2024,"2,524.10","2,528.90","2,534.00","2,505.60",5.72K,-0.13%


Dropping unecesssary columns

In [6]:
gold = gold[['Date','GoldPrice','GoldChange %']]
gold.head()

,Date,GoldPrice,GoldChange %
0,27-08-2024,"2,529.30",-0.08%
1,26-08-2024,"2,531.40",0.35%
2,23-08-2024,"2,522.60",1.17%
3,22-08-2024,"2,493.50",-1.21%
4,21-08-2024,"2,524.10",-0.13%


change columns to right data type

In [7]:
gold['Date']=pd.to_datetime(gold['Date'])

gold['GoldPrice']=gold['GoldPrice'].astype(str).str.replace(',','')
gold['GoldPrice']=gold['GoldPrice'].astype(float)

gold['GoldChange %']=gold['GoldChange %'].astype(str).str.replace("%","")
gold['GoldChange %']=gold['GoldChange %'].astype(float)

gold.info()
gold

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4202 entries, 0 to 4201
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          4202 non-null   datetime64[ns]
 1   GoldPrice     4202 non-null   float64       
 2   GoldChange %  4202 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 98.6 KB


C:\Users\Asus\AppData\Local\Temp\ipykernel_26784\2664188883.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  gold['Date']=pd.to_datetime(gold['Date'])


,Date,GoldPrice,GoldChange %
0,2024-08-27,2529.3,-0.08
1,2024-08-26,2531.4,0.35
2,2024-08-23,2522.6,1.17
3,2024-08-22,2493.5,-1.21
4,2024-08-21,2524.1,-0.13
...,...,...,...
4197,2008-03-10,971.8,-0.25
4198,2008-03-07,974.2,-0.30
4199,2008-03-06,977.1,-1.15
4200,2008-03-05,988.5,2.30


Date i.e, index is not in accending order so we will correct it 

In [8]:
gold = gold.iloc[::-1].reset_index(drop=True)
gold.head()

,Date,GoldPrice,GoldChange %
0,2008-03-04,966.3,-1.82
1,2008-03-05,988.5,2.30
2,2008-03-06,977.1,-1.15
3,2008-03-07,974.2,-0.30
4,2008-03-10,971.8,-0.25


---

## **Crude Oil**

In [9]:
df2 = pd.read_csv("MacroData/Crudeoil.csv")
crude = df2.copy()
crude.head()

,Date,CrudePrice,CrudeOpen,CrudeHigh,CrudeLow,Vol.,CrudeChange %
0,03/04/2008,99.52,102.35,103.33,98.87,366.72K,-2.86%
1,03/05/2008,104.52,99.88,104.95,99.55,414.31K,5.02%
2,03/06/2008,105.47,104.64,105.97,102.85,337.77K,0.91%
3,03/07/2008,105.15,105.58,106.54,103.91,292.40K,-0.30%
4,03/10/2008,107.90,105.25,108.21,104.08,340.55K,2.62%


Dropping uncessecary columns

In [10]:
crude = crude[["Date","CrudePrice","CrudeChange %"]]

Changing columns to right datatype

In [11]:
crude['Date']=pd.to_datetime(crude['Date'])

crude['CrudePrice']=crude['CrudePrice'].astype(str).str.replace(',','')
crude['CrudePrice']=crude['CrudePrice'].astype(float)

crude['CrudeChange %']=crude['CrudeChange %'].astype(str).str.replace("%","")
crude['CrudeChange %']=crude['CrudeChange %'].astype(float)

crude.info()
crude

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4281 entries, 0 to 4280
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           4281 non-null   datetime64[ns]
 1   CrudePrice     4281 non-null   float64       
 2   CrudeChange %  4281 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 100.5 KB


,Date,CrudePrice,CrudeChange %
0,2008-03-04,99.52,-2.86
1,2008-03-05,104.52,5.02
2,2008-03-06,105.47,0.91
3,2008-03-07,105.15,-0.30
4,2008-03-10,107.90,2.62
...,...,...,...
4276,2024-08-21,71.30,-2.56
4277,2024-08-22,72.28,1.37
4278,2024-08-23,73.93,2.28
4279,2024-08-26,76.17,3.03


---

## **USDINR**

In [12]:
df3 = pd.read_csv("MacroData/usdinr.csv")
usdinr = df3.copy()
usdinr.head()

,Date,inrPrice,inrOpen,inrHigh,inrLow,Vol.,inrChange %
0,08/27/2024,83.910,83.865,83.950,83.865,NaN,0.08%
1,08/26/2024,83.840,83.801,83.910,83.776,NaN,0.04%
2,08/23/2024,83.809,83.913,83.933,83.778,NaN,-0.14%
3,08/22/2024,83.930,83.895,83.965,83.876,NaN,0.07%
4,08/21/2024,83.870,83.791,83.935,83.726,NaN,0.14%


Dropping uncessecary columns

In [13]:
usdinr = usdinr[["Date","inrPrice","inrChange %"]]

Changing columns to right datatype

In [14]:
usdinr['Date']=pd.to_datetime(usdinr['Date'])

usdinr['inrPrice']=usdinr['inrPrice'].astype(str).str.replace(',','')
usdinr['inrPrice']=usdinr['inrPrice'].astype(float)

usdinr['inrChange %']=usdinr['inrChange %'].astype(str).str.replace("%","")
usdinr['inrChange %']=usdinr['inrChange %'].astype(float)

usdinr.info()
usdinr

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301 entries, 0 to 4300
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4301 non-null   datetime64[ns]
 1   inrPrice     4301 non-null   float64       
 2   inrChange %  4301 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 100.9 KB


,Date,inrPrice,inrChange %
0,2024-08-27,83.910,0.08
1,2024-08-26,83.840,0.04
2,2024-08-23,83.809,-0.14
3,2024-08-22,83.930,0.07
4,2024-08-21,83.870,0.14
...,...,...,...
4296,2008-03-10,40.400,-0.07
4297,2008-03-07,40.430,0.72
4298,2008-03-06,40.140,-0.15
4299,2008-03-05,40.200,-0.05


---

In [15]:
df4 = pd.read_csv("MacroData/USdollarindex.csv")
usdindex = df4.copy()
usdindex.head()

,Date,diPrice,diOpen,diHigh,diLow,Vol.,diChange %
0,27-08-2024,100.55,100.82,100.93,100.51,NaN,-0.30%
1,26-08-2024,100.85,100.68,100.92,100.53,NaN,0.13%
2,23-08-2024,100.72,101.46,101.55,100.60,NaN,-0.78%
3,22-08-2024,101.51,101.13,101.63,101.09,NaN,0.46%
4,21-08-2024,101.04,101.35,101.63,100.92,NaN,-0.40%


Dropping uncessecary columns

In [16]:
usdindex = usdindex[["Date","diPrice","diChange %"]]

Changing columns to right datatype

In [17]:
usdindex['Date']=pd.to_datetime(usdindex['Date'])

usdindex['diPrice']=usdindex['diPrice'].astype(str).str.replace(',','')
usdindex['diPrice']=usdindex['diPrice'].astype(float)

usdindex['diChange %']=usdindex['diChange %'].astype(str).str.replace("%","")
usdindex['diChange %']=usdindex['diChange %'].astype(float)

usdindex.info()
usdindex

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        4298 non-null   datetime64[ns]
 1   diPrice     4298 non-null   float64       
 2   diChange %  4298 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 100.9 KB


C:\Users\Asus\AppData\Local\Temp\ipykernel_26784\2305087138.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  usdindex['Date']=pd.to_datetime(usdindex['Date'])


,Date,diPrice,diChange %
0,2024-08-27,100.55,-0.30
1,2024-08-26,100.85,0.13
2,2024-08-23,100.72,-0.78
3,2024-08-22,101.51,0.46
4,2024-08-21,101.04,-0.40
...,...,...,...
4293,2008-03-10,72.99,-0.05
4294,2008-03-07,73.03,0.04
4295,2008-03-06,73.00,-0.65
4296,2008-03-05,73.48,-0.24


---

## **10-2year bond yeild**  

In [31]:
df5 = pd.read_csv("MacroData/10-2year.csv")


In [32]:
df5["Date"] = pd.to_datetime(df5['Date'])
df5.info()
df5.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301 entries, 0 to 4300
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    4301 non-null   datetime64[ns]
 1   T10Y2Y  4301 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 67.3+ KB


,Date,T10Y2Y
0,2008-03-04,1.98
1,2008-03-05,2.04
2,2008-03-06,2.09
3,2008-03-07,2.03
4,2008-03-10,1.99
5,2008-03-11,1.86
6,2008-03-12,1.86
7,2008-03-13,1.93
8,2008-03-14,1.97
9,2008-03-17,1.99


converting **T10Y2Y** column from object type to float

In [33]:
df5["T10Y2Y"] = df5['T10Y2Y'].astype(float)

df5.info()

ValueError: could not convert string to float: '.'

There may be multiple '.' values in the column let us convert them to NaN


In [34]:
df5['T10Y2Y']=df5['T10Y2Y'].replace(".", np.nan)

"." is converted to NaN successfully. 



In [38]:
df5[12:14]

,Date,T10Y2Y
12,2008-03-20,1.75
13,2008-03-21,NaN


Now we will fill the NaN values with preceding values and check if it's working

In [40]:
df5['T10Y2Y'] = df5['T10Y2Y'].fillna(method='ffill')    # method = "ffill" fills NaN values with previous values

C:\Users\Asus\AppData\Local\Temp\ipykernel_26784\1687390286.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df5['T10Y2Y'] = df5['T10Y2Y'].fillna(method='ffill')    # method = "ffill" fills NaN values with previous values


In [41]:
print(df5['T10Y2Y'].isna().sum())  # This should output 0 if all NaNs are filled.


0


All NaN values are filled with preceding values succcessfully

In [42]:
df5["T10Y2Y"] = df5['T10Y2Y'].astype(float)

In [44]:
df5[12:14]

,Date,T10Y2Y
12,2008-03-20,1.75
13,2008-03-21,1.75


Adding **"percent change"** as a feature

In [45]:
df5['T10Y2Y%chng'] = df5['T10Y2Y'].pct_change() * 100 
T10Y2Ydf = df5.copy()
T10Y2Ydf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4301 entries, 0 to 4300
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         4301 non-null   datetime64[ns]
 1   T10Y2Y       4301 non-null   float64       
 2   T10Y2Y%chng  4298 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 100.9 KB


In [47]:
T10Y2Ydf.head()

,Date,T10Y2Y,T10Y2Y%chng
0,2008-03-04,1.98,NaN
1,2008-03-05,2.04,3.030303
2,2008-03-06,2.09,2.450980
3,2008-03-07,2.03,-2.870813
4,2008-03-10,1.99,-1.970443


There is a NaN value in first row, we will make it zero

In [ ]:
df5["T10Y2Y%chng"] = df5['T10Y2Y%chng'].fillna('0')
df5

,Date,T10Y2Y,T10Y2Y%chng
0,2008-03-04,1.98,0
1,2008-03-05,2.04,3.030303
2,2008-03-06,2.09,2.45098
3,2008-03-07,2.03,-2.870813
4,2008-03-10,1.99,-1.970443
...,...,...,...
4296,2024-08-21,-0.13,-23.529412
4297,2024-08-22,-0.13,0.0
4298,2024-08-23,-0.09,-30.769231
4299,2024-08-26,-0.09,0.0


---

all the DataFrames above were with daily frequency, Now below we will deal with the DataFrames that were not in daily frequency 

In [ ]:
# code to merge DFs accordint to index 
import pandas as pd

# Load CSV files into DataFrames
indiavix_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\indiavix.csv', parse_dates=['Date'])
gold_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\Gold.csv', parse_dates=['Date'])
crude_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\Crudeoil.csv', parse_dates=['Date'])
usdinr_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\usdinr.csv', parse_dates=['Date'])
usdindex_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\USdollarindex.csv', parse_dates=['Date'])
tenmtwo_df = pd.read_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\tenMtwo.csv', parse_dates=['Date'])

# Set 'Date' column as the index for each DataFrame
indiavix_df.set_index('Date', inplace=True)
gold_df.set_index('Date', inplace=True)
crude_df.set_index('Date', inplace=True)
usdinr_df.set_index('Date', inplace=True)
usdindex_df.set_index('Date', inplace=True)
tenmtwo_df.set_index('Date', inplace=True)

# Merge all DataFrames, using IndiaVix as the reference and preserving its index
# This ensures that rows not present in IndiaVix will be dropped
merged_df = indiavix_df

# Merge each DataFrame one by one, ensuring 'Date' index is maintained
merged_df = merged_df.join(gold_df, how='left', rsuffix='_gold')
merged_df = merged_df.join(crude_df, how='left', rsuffix='_crude')
merged_df = merged_df.join(usdinr_df, how='left', rsuffix='_usdinr')
merged_df = merged_df.join(usdindex_df, how='left', rsuffix='_usdindex')
merged_df = merged_df.join(tenmtwo_df, how='left', rsuffix='_tenmtwo')

# Forward fill missing values where the index exists in IndiaVix but not in other DataFrames
merged_df.ffill(inplace=True)

# Check the result
print(merged_df.head())

# Save the merged DataFrame to CSV if needed
merged_df.to_csv(r'C:\Users\Asus\Desktop\Nifty PredictionbyMacro\MacroData\merged_data.csv', index=True)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Asus\\Desktop\\Nifty PredictionbyMacro\\MacroData\\indiavix.csv'